In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
intents = [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello"],
     "responses": ["Hello", "Hi", "Hi there"]
     },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
     },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
     },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?"],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
     },
    {"tag": "name",
     "patterns": ["what is your name", "what should I call you", "whats your name?"],
     "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
     },
    {"tag": "help",
     "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
     "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
     },
    {"tag": "createaccount",
     "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
     "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
     },
    {"tag": "complaint",
     "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
     "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
     }
]

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in intents:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [7]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 11ms/step - loss: 2.0794 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0775 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0765 - accuracy: 0.1515
Epoch 4/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0754 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0744 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0734 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0727 - accuracy: 0.1515
Epoch 8/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0718 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0711 - accuracy: 0.1515
Epoch 10/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0704 - accuracy: 0.1515
Epoch 11/500
2/2 [==========

In [8]:
def chat():
    # parameters
    max_len = 20

    while True:
        print("User: ", end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                                          truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in intents:
            if i['tag'] == tag:
                print("ChatBot:", np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))


print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!
User: ChatBot: Hi
User: ChatBot: Any time!
User: ChatBot: Hi
User: ChatBot: You're most welcome!
User: ChatBot: You can call me Joana.
User: ChatBot: You can call me Joana.
User: ChatBot: Happy to help!
User: ChatBot: Hello
User: ChatBot: Any time!
User: ChatBot: Any time!
User: ChatBot: Happy to help!
User: ChatBot: My pleasure
User: ChatBot: You're most welcome!
User: ChatBot: Any time!
User: ChatBot: Happy to help!
User: ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: ChatBot: Hi there
User: ChatBot: Please provide us your complaint in order to assist you
User: ChatBot: You can call me Joana.
User: 